# Programacion Evolutiva
## Eduardo Manuel Ceja Cruz

In [1]:
import sys
import os
import math
import numpy as np

from typing import Tuple, List

Usando programacion evolutiva, resolver el siguente probelma:

$$
\min f(\vec{x}) = -20exp \left( -0.2 \sqrt{\frac{1}{n} \sum_{i=1}^n x^2_i} \right) - exp \left(\frac{1}{n}\sum_{i=1}^n cos(2\pi x_i) \right) + 20 + e
$$

In [2]:
MAX_DOMAIN = 32
MIN_DOMAIN = -32

In [3]:
def read_input(filename : str = "input.txt") -> Tuple[int, int, float, float]:
    try:
        with open(filename) as f:
            lines = f.readlines()
    except FileNotFoundError:
        print(f"El archivo {filename} no fue encontrado")
        return
    except Exception as e:
        print(f"ha ocurrido un problema {e}")
        return
    
    n = int(lines.pop(0))
    mu, g = map(int, lines.pop(0).split())
    alpha, epsilon = map(float, lines.pop(0).split())
    return n,mu,g,alpha,epsilon

In [4]:
def std(M_list: list, mean: float) -> float:
    return np.sqrt(sum([(sol_i[-1] - mean)**2 for sol_i in M_list])/len(M_list))

In [5]:
def f(x : List[float], n : int) -> float:
    coso1 = -0.2*np.sqrt(sum([xi**2 for xi in x[:n]])/n)
    coso2 = (sum([math.cos(2*math.pi*xi) for xi in x[:n]])/n)
    return -20*math.e**(coso1) -math.e**(coso2) + 20 + math.e

In [6]:
def initial_population(mu : int, n : int) -> List[List[float]]:
    padres = []
    for i in range(mu):
        p = np.concatenate([np.random.uniform(MIN_DOMAIN, MAX_DOMAIN, n) ,np.random.uniform(0,1, n)])
        p = np.concatenate([p, [f(p, n)]])
        padres.append(p)
    return padres

In [7]:
parents = initial_population(100,2); parents

[array([ -4.92962148, -30.14443428,   0.75115522,   0.39014209,
         20.31443591]),
 array([ 24.51597757, -23.13117567,   0.70709031,   0.47983523,
         21.69419226]),
 array([-1.79597822e+01, -2.66140363e+01,  2.46300699e-02,  3.46623870e-01,
         2.13919333e+01]),
 array([16.48088114, -2.10148655,  0.80677501,  0.49903652, 19.90048298]),
 array([17.70868004, 27.57569112,  0.99198615,  0.09884351, 21.96019444]),
 array([-17.59099095, -20.43569483,   0.33334059,   0.22602862,
         21.86208646]),
 array([ -7.4007909 , -29.74704583,   0.13582983,   0.3216827 ,
         21.79609383]),
 array([-10.98910556,   7.57188539,   0.59400086,   0.43677274,
         18.63846994]),
 array([-16.91946526, -22.11416341,   0.92937362,   0.47781252,
         20.07124108]),
 array([-13.09047795, -24.30482422,   0.16709379,   0.58385508,
         21.02783471]),
 array([12.7646206 , 12.40618101,  0.42047466,  0.25841769, 20.41388692]),
 array([-25.95219102, -20.05683354,   0.52996255,   0.31

In [8]:
parents[0][2:-1] * (100 *np.random.normal(0,1,2))

array([-144.84607641,  -33.66162517])

In [14]:
def create_individual(parent : List[float], n: int, alpha : float, epsilon : float):
    # print(parent.shape)
    offspring_sigma = parent[n:-1]*(1 + (alpha *np.random.normal(0,1,n)))
    offspring_sigma[offspring_sigma < epsilon] = epsilon

    offspring_x = parent[:n] + (offspring_sigma* np.random.normal(0,1, n))
    offspring_x[offspring_x < MIN_DOMAIN] = MIN_DOMAIN
    offspring_x[offspring_x > MAX_DOMAIN] = MAX_DOMAIN

    return np.concatenate([offspring_sigma, offspring_x])

In [15]:
np.concatenate ([np.random.uniform(-1, 1, 3), np.random.uniform(0,1, 3)])

array([-0.33308288, -0.44293787,  0.88592518,  0.4350615 ,  0.37409303,
        0.38661875])

In [16]:
def programacion_evolutiva(n : int,mu : int,g : int ,alpha : float,epsilon : float):
    padres = initial_population(mu, n)
    t = 1
    while t <= g:
        current_gen = padres.copy()
        for p in padres:
            offspring = create_individual(p, n, alpha, epsilon)
            current_gen.append(np.concatenate([offspring, [f(offspring, n)]]))
        
        padres = sorted(current_gen, key = lambda x : x[-1])[:mu]
        t+=1
    return padres[0]    #la mejor sol

In [17]:
programacion_evolutiva(*read_input())

array([ 1.00000000e-04,  1.00000000e-04, -4.54375476e+00,  2.91953053e+01,
        4.00532567e-04])

In [18]:
f([1.00000000e-04,  1.00000000e-04,  2.24971042e+01, -5.69032933e+00], 2)

0.00040053256728400655

In [19]:
def statistical(M : int = 10, filename : str = 'input.txt'):
    # n,mu,g,alpha,epsilon
    args = read_input(filename)
    sols = []
    for _ in range(M):
        sols.append(programacion_evolutiva(*args))
    
    sols = sorted(sols, key = lambda x : x[-1], reverse=True)
    print(f"Besto sol : {sols[0]}")
    print(f"Worsto sol: {sols[-1]}")
    print(f"Mediana : {sols[M//2]}")
    mean = sum([_[-1] for _ in sols])/M
    desviacion = std(sols, mean)
    print(f"Media {mean}")
    print(f"STD : {desviacion}")

In [20]:
statistical() #2 varaibles

Besto sol : [1.00000000e-04 1.00000000e-04 1.46274211e+00 2.30266099e+01
 4.00532567e-04]
Worsto sol: [ 1.00000000e-04  1.00000000e-04 -2.34076587e+01 -7.00968019e+00
  4.00532567e-04]
Mediana : [ 1.00000000e-04  1.00000000e-04 -1.40140993e+01  5.73075915e+00
  4.00532567e-04]
Media 0.00040053256728400655
STD : 0.0


In [21]:
statistical(filename="input10.txt") # 10 variables

Besto sol : [1.00000000e-04 1.00000000e-04 1.00000000e-04 1.00000000e-04
 1.00000000e-04 1.00000000e-04 1.00000000e-04 1.00000000e-04
 1.00000000e-04 1.00000000e-04 2.12363425e-01 6.33064864e-01
 1.04919499e+00 2.17119509e-01 2.14514821e-01 9.25225265e-02
 2.90600603e-01 1.67582695e+00 1.09318716e-01 1.31068377e-04
 4.00532567e-04]
Worsto sol: [ 1.00000000e-04  1.00000000e-04  1.00000000e-04  1.00000000e-04
  1.00000000e-04  1.00000000e-04  1.00000000e-04  1.00000000e-04
  1.00000000e-04  1.00000000e-04  4.93622442e-05  2.24663418e-01
  2.24348632e-01 -7.83586284e-05  3.68100901e-01  8.94478210e-05
  1.14242717e+00  1.03325578e+00  2.39016938e+00  1.04570232e-01
  4.00532567e-04]
Mediana : [ 1.00000000e-04  1.00000000e-04  1.00000000e-04  1.00000000e-04
  1.00000000e-04  1.00000000e-04  1.00000000e-04  1.00000000e-04
  1.00000000e-04  1.00000000e-04  1.20976715e+00 -7.71715820e-06
  1.36677248e-04  3.61094144e-02  2.66831956e-05  2.22849155e-04
  2.43213533e-04  5.86503543e-03  1.28514

In [22]:
statistical(filename="input20.txt") # 20 variables

Besto sol : [ 1.00000000e-04  4.48068943e-04  1.11638502e-04  1.00000000e-04
  1.00000000e-04  1.00000000e-04  1.00000000e-04  1.00000000e-04
  1.00000000e-04  1.00000000e-04  1.00000000e-04  1.00000000e-04
  1.00000000e-04  1.00000000e-04  1.00000000e-04  1.00000000e-04
  1.00000000e-04  2.52255741e-04  1.00000000e-04  1.25387780e-04
 -6.38813984e-05  8.18081740e-04  1.32536596e-04  2.03121932e-04
  7.21888787e-05  4.53746733e-05  3.62114332e-04  3.36193211e-04
  2.21804848e-04  9.73888799e-05  1.12037904e-04  7.40406500e-04
  7.26339590e-04  1.16506562e-04 -2.26496424e-06  7.53405947e-04
  1.18713226e-04  2.19644867e-05  1.06367201e-03  5.55669505e-04
  6.02925349e-04]
Worsto sol: [ 1.00000000e-04  1.00000000e-04  1.00000000e-04  1.00000000e-04
  1.00000000e-04  1.00000000e-04  1.00000000e-04  1.00000000e-04
  1.00000000e-04  1.00000000e-04  1.00000000e-04  1.00000000e-04
  1.00000000e-04  1.00000000e-04  1.00000000e-04  1.00000000e-04
  1.00000000e-04  1.00000000e-04  1.00000000e-04